In [2]:
import os
import pandas as pd
import numpy as np
from datetime import timedelta, datetime
import datetime as dt
import pandas_ta as ta
import yfinance as yf
import copy
import random
from pymongo import MongoClient
import sys
from dotenv import load_dotenv
import warnings

warnings.filterwarnings("ignore")
warnings.simplefilter('ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

load_dotenv()
mongo_client = MongoClient(os.environ.get("MONGO_URI"))
stock_db = mongo_client["stock_db"]
ref_db = mongo_client["ref_db"]

In [3]:
full_stock_classification = pd.read_excel("../.xlsx/full_stock_classification.xlsx", sheet_name='classification')
name_map = pd.read_excel("../.xlsx/full_stock_classification.xlsx", sheet_name='name_map').drop(columns=['group', 'order'],axis=1)
order_map = pd.read_excel("../.xlsx/full_stock_classification.xlsx", sheet_name='name_map').drop(columns=['group', 'full_name'],axis=1)
group_map = pd.read_excel("../.xlsx/full_stock_classification.xlsx", sheet_name='name_map').drop(columns=['order', 'full_name'],axis=1)

itd_time_percent = pd.read_excel("../.xlsx/itd_time_percent.xlsx", sheet_name='itd_time_percent')
quarter_stock_map = pd.read_excel("../.xlsx/quarter_stock_map.xlsx", sheet_name='quarter_stock_list')
quarter_map = pd.read_excel("../.xlsx/quarter_stock_map.xlsx", sheet_name='quarter_map')

In [4]:
def overwrite_mongo(collection, df):
    # Lấy tên collection hiện tại và database
    collection_name = collection.name
    db = collection.database  # Truy cập database từ collection
    temp_collection_name = f"temp_{collection_name}"
    old_collection_name = f"old_{collection_name}"

    # Reset index của DataFrame
    df = df.reset_index(drop=True)
    records = df.replace({pd.NaT: None}).to_dict(orient='records')

    # 1. Lưu dữ liệu vào collection tạm
    temp_collection = db[temp_collection_name]
    temp_collection.drop()  # Đảm bảo collection tạm sạch trước khi insert
    temp_collection.insert_many(records)

    # 2. Rename collection cũ thành 'old_' (nếu tồn tại)
    if collection_name in db.list_collection_names():
        db[collection_name].rename(old_collection_name, dropTarget=True)

    # 3. Rename collection tạm thành tên chuẩn
    temp_collection.rename(collection_name, dropTarget=True)

    # 4. Xóa collection 'old_' (nếu tồn tại)
    if old_collection_name in db.list_collection_names():
        db[old_collection_name].drop()
        
overwrite_mongo(ref_db["full_stock_classification"], full_stock_classification)
overwrite_mongo(ref_db["name_map"], name_map)
overwrite_mongo(ref_db["order_map"], order_map)
overwrite_mongo(ref_db["group_map"], group_map)
overwrite_mongo(ref_db["itd_time_percent"], itd_time_percent)
overwrite_mongo(ref_db["quarter_stock_map"], quarter_stock_map)
overwrite_mongo(ref_db["quarter_map"], quarter_map)